In [1]:
import casadi as ca
import sympy as sp
from sympy import symbols, Matrix

# Define SymPy symbols
theta1_sp, theta2_sp = sp.symbols('theta1 theta2')
l1_sp, l2_sp = sp.symbols('l1 l2')


# Define the forward kinematics equations for SymPy
xe_sp = l1_sp * sp.cos(theta1_sp) + l2_sp * sp.cos(theta1_sp + theta2_sp)
ye_sp = l1_sp * sp.sin(theta1_sp) + l2_sp * sp.sin(theta1_sp + theta2_sp)

# Combine into a vector for SymPy
end_effector_sp = Matrix([xe_sp, ye_sp])

# Compute the Jacobian matrix using SymPy
jacobian_sp = end_effector_sp.jacobian([theta1_sp, theta2_sp])

# Compute the inverse of the Jacobian matrix using SymPy
jacobian_inv_sym = jacobian_sp.inv()

# Print the Jacobian matrix and its inverse
print("Jacobian matrix (SymPy):\n")
sp.pprint(jacobian_sp)
print("\n\nInverse Jacobian matrix (SymPy):\n")
sp.pprint(jacobian_inv_sym)


Jacobian matrix (SymPy):

⎡-l₁⋅sin(θ₁) - l₂⋅sin(θ₁ + θ₂)  -l₂⋅sin(θ₁ + θ₂)⎤
⎢                                               ⎥
⎣l₁⋅cos(θ₁) + l₂⋅cos(θ₁ + θ₂)   l₂⋅cos(θ₁ + θ₂) ⎦


Inverse Jacobian matrix (SymPy):

⎡                     -cos(θ₁ + θ₂)                                           
⎢   ─────────────────────────────────────────────────         ────────────────
⎢   l₁⋅sin(θ₁)⋅cos(θ₁ + θ₂) - l₁⋅sin(θ₁ + θ₂)⋅cos(θ₁)         l₁⋅sin(θ₁)⋅cos(θ
⎢                                                                             
⎢             -l₁⋅cos(θ₁) - l₂⋅cos(θ₁ + θ₂)                             -l₁⋅si
⎢────────────────────────────────────────────────────────  ───────────────────
⎣-l₁⋅l₂⋅sin(θ₁)⋅cos(θ₁ + θ₂) + l₁⋅l₂⋅sin(θ₁ + θ₂)⋅cos(θ₁)  -l₁⋅l₂⋅sin(θ₁)⋅cos(

  -sin(θ₁ + θ₂)                      ⎤
─────────────────────────────────    ⎥
₁ + θ₂) - l₁⋅sin(θ₁ + θ₂)⋅cos(θ₁)    ⎥
                                     ⎥
n(θ₁) - l₂⋅sin(θ₁ + θ₂)              ⎥
─────────────────────────────────────⎥


In [2]:
# Define CasADi MX symbols
theta1 = ca.MX.sym('theta1')
theta2 = ca.MX.sym('theta2')
l1 = ca.MX.sym('l1')
l2 = ca.MX.sym('l2')

# Define the forward kinematics equations for CasADi
xe_ca = l1 * ca.cos(theta1) + l2 * ca.cos(theta1 + theta2)
ye_ca = l1 * ca.sin(theta1) + l2 * ca.sin(theta1 + theta2)

# Combine into a vector
end_effector_ca = ca.vertcat(xe_ca, ye_ca)

# Compute the Jacobian matrix using CasADi
jacobian_ca = ca.jacobian(end_effector_ca, ca.vertcat(theta1, theta2))

# Create a CasADi function for the Jacobian 
J_func = ca.Function('J_func', [theta1, theta2, l1,l2], [jacobian_ca])

# Compute the inverse of the Jacobian matrix using CasADi
jacobian_inv_ca = ca.inv(jacobian_ca)

# Create a CasADi function for the Jacobian inverse
jacobian_inv_func = ca.Function('jacobian_inv_ca', [theta1, theta2, l1, l2], [jacobian_inv_ca])


In [3]:
# Example usage with CasADi
theta1_degree = 0.01      # Initial values
theta2_degree = 179.9      # Initial values

#converted to radian
theta1_val = ((theta1_degree * ca.pi)/(180))
theta2_val = ((theta2_degree * ca.pi)/(180))
l1_val = 14.0
l2_val = 15.0

print(theta1_val)
print(theta2_val)


0.00017453292519943296
3.139847324337799


In [4]:
# Request user input for the next position
xe_2 = float(input("\nEnter the Target x-coordinate of the next position: "))
ye_2 = float(input("Enter the Target y-coordinate of the next position: "))
new_position = ca.vertcat(xe_2, ye_2)

print(f"\nTarget position: ({xe_2}, {ye_2})")


Enter the Target x-coordinate of the next position: 12
Enter the Target y-coordinate of the next position: 9

Target position: (12.0, 9.0)


In [5]:
# Iterative loop to update position
tolerance = 1e-7  # Define a tolerance for convergence
max_iterations = 200  # Define a maximum number of iterations (epochs)
learning_rate = 0.00001  # Define a learning rate
iteration = 0

In [6]:

#num_iterations = 20

while iteration < max_iterations:
    # Compute the current end effector position
    current_position = ca.vertcat(
        l1_val * ca.cos(theta1_val) + l2_val * ca.cos(theta1_val + theta2_val),
        l1_val * ca.sin(theta1_val) + l2_val * ca.sin(theta1_val + theta2_val)
    )
    #print(f"\n\n\n\nCurrent position : ({current_position})")  #check text
    
    
    #dx is (2x1)
    dx = new_position - current_position
    print(f"\n\n\n\n\npos_diff: {dx}")
    
    #Covert Inverse function into a Dense Matrix or DM
    jacobian_inv_val = jacobian_inv_func(theta1_val, theta2_val, l1_val, l2_val)
    print(f"\nJacobian inverse function: {jacobian_inv_val}")

    # dq is (2x1)
    dq = jacobian_inv_val @ dx
    print(f"\ndq: {dq}")
    
    #updating previous position of angles
    theta1_val = theta1_val + dq[0]
    theta2_val = theta2_val + dq[1]
    
    THETAx = ca.vertcat(theta1_val, theta2_val)
    print(f"\nCurrent_angles: {THETAx}")
    
    print(f"\nCurrent position: ({current_position})")
    
    # Check for convergence
    if ca.norm_2(dx) < tolerance:
        print("\nConvergence achieved")
        break

    iteration += 1

if iteration >= max_iterations:
    print("\nMax iterations reached. Convergence not achieved.")

print(f"\n\n\nFinal joint angles: {theta1_val, theta2_val}")
print(f"Final end effector position: {new_position}")
    






pos_diff: [13, 8.97399]

Jacobian inverse function: 
[[-40.9255, 0.0642857], 
 [2.72832, -0.0709524]]

dq: [-531.454, 34.8314]

Current_angles: [-531.454, 37.9713]

Current position: ([-0.999982, 0.0260054])





pos_diff: [38.639, -1.76609]

Jacobian inverse function: 
[[-0.257293, 0.0664453], 
 [0.471925, -0.190728]]

dq: [-10.0589, 18.5715]

Current_angles: [-541.513, 56.5428]

Current position: ([-26.639, 10.7661])





pos_diff: [0.472525, 35.6103]

Jacobian inverse function: 
[[-4.79005, 11.1442], 
 [9.32193, -21.519]]

dq: [394.586, -761.894]

Current_angles: [-146.927, -705.351]

Current position: ([11.5275, -26.6103])





pos_diff: [31.6982, 6.51017]

Jacobian inverse function: 
[[0.0441375, -0.0563449], 
 [-0.0939923, 0.0118805]]

dq: [1.03227, -2.90205]

Current_angles: [-145.894, -708.253]

Current position: ([-19.6982, 2.48983])





pos_diff: [-4.65006, 17.3871]

Jacobian inverse function: 
[[0.0677535, 0.0259303], 
 [-0.0805263, 0.0405636]]

dq: [0.135797, 1.07974]